## Initialize Dependencies

In [1]:
from csv import DictReader
import os
import urllib
from csv import DictReader
from urllib.parse import urljoin
from utilities import init_db_handle, init_keys, format_record, is_empty_record
from datetime import datetime

## Download Austin Tx 311 Data
-[Austin Tx 311 Data Description](https://data.austintexas.gov/Utilities-and-City-Services/311-Unified-Data/i26j-ai4z)  
-[Download *.csv file from website](https://stackoverflow.com/questions/41992223/download-csv-from-web-service-with-python-3)  
-[Check for file existence](https://stackoverflow.com/questions/23459095/check-for-file-existence-in-python-3)

In [2]:
csv_url = urljoin('https://data.austintexas.gov/api',
                  'views/i26j-ai4z/rows.csv?accessType=DOWNLOAD')

csv_file = './Data/austinTx311Data.csv'
    
if not os.path.isfile(csv_file):
    data_dir = os.path.split(csv_file)[0]

    if not os.path.exists(data_dir):
        os.mkdir(data_dir)

    urllib.request.urlretrieve(csv_url, csv_file)

## Initialize Couchbase interface
-[Couchbase Python SDK example](https://developer.couchbase.com/documentation/server/current/sdk/python/start-using-sdk.html)

In [3]:
cb = init_db_handle(os.path.split(csv_file)[0])

In [ ]:
with open(csv_file, 'rt') as h_csv:
    keys = init_keys(h_csv)

    readerobj = DictReader(h_csv, keys)

    for record in readerobj:
        if not is_empty_record(record):
            record = format_record(record)
            cb.upsert(record[0], record[1])

## Create Primary Index
-[Couchbase Python SDK Hello World](https://developer.couchbase.com/documentation/server/current/sdk/python/start-using-sdk.html)

In [ ]:
cb.n1ql_query('CREATE PRIMARY INDEX ON AustinTx311Data').execute()